In [1]:
!pip install konlpy
!pip install soynlp
!pip install transformers
!pip install git+https://github.com/ssut/py-hanspell.git
!pip install 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 50.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 8.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.5 MB/s 
     |████████████████████████████████| 86 kB 2.8 MB/s 
     |████████████████████████████████| 596 kB 43.5 MB/s 
     |████████████████████████████████| 6.6 MB 39.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to 

In [2]:
import pandas as pd
import numpy as np

import re
from konlpy.tag import Okt
from konlpy.tag import Hannanum
from hanspell import spell_checker
from tensorflow.keras.preprocessing.text import Tokenizer

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from wordcloud import WordCloud
from collections import Counter

## 데이터 불러오기 및 확인

In [4]:
youtube_df = pd.read_csv("preprocessed_emotion_labeled_youtube_0607.csv", lineterminator='\n')
youtube_df.head()

,car_brand,car_model,comments,likes,related,emotion
0,BMW,1시리즈,"주제 선정 좋아요. 더운 여름 잘 지내세요. 비싼 차, 싼 차, 슈퍼카, 경차 가리...",3,1,0
1,BMW,1시리즈,주차장시승평잘봤네요ㅋ그냥외관이랑실내평만보면편안할듯ㅋㅋ매력적인차네요세컨으로,3,1,0
2,BMW,1시리즈,차 가격도 싸고 디자인도 예쁘고 엄청 좋다,5,1,0
3,BMW,1시리즈,좀끄럽다하셨는데G305시리즈랑어느정도차이가날까요?ㅎㅎ,2,1,0
4,BMW,1시리즈,그나마 전륜으로 바뀌면서 공간이 좀 쓸 만해졌죠. 이 전 세대 1시리즈는 정말......,2,1,1


In [5]:
youtube_df.rename(columns = {'likes\r':'likes'}, inplace = True) # 저는 파일을 읽어오면 likes\r로 columns가 나와서 에러가 나나 싶어소 혹시 바꾸어줬습니다.

In [6]:
youtube_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120488 entries, 0 to 120487
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   car_brand  120488 non-null  object
 1   car_model  120488 non-null  object
 2   comments   120183 non-null  object
 3   likes      120488 non-null  int64 
 4   related    120488 non-null  int64 
 5   emotion    120488 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 5.5+ MB


In [7]:
youtube_df

,car_brand,car_model,comments,likes,related,emotion
0,BMW,1시리즈,"주제 선정 좋아요. 더운 여름 잘 지내세요. 비싼 차, 싼 차, 슈퍼카, 경차 가리...",3,1,0
1,BMW,1시리즈,주차장시승평잘봤네요ㅋ그냥외관이랑실내평만보면편안할듯ㅋㅋ매력적인차네요세컨으로,3,1,0
2,BMW,1시리즈,차 가격도 싸고 디자인도 예쁘고 엄청 좋다,5,1,0
3,BMW,1시리즈,좀끄럽다하셨는데G305시리즈랑어느정도차이가날까요?ㅎㅎ,2,1,0
4,BMW,1시리즈,그나마 전륜으로 바뀌면서 공간이 좀 쓸 만해졌죠. 이 전 세대 1시리즈는 정말......,2,1,1
...,...,...,...,...,...,...
120483,혼다,파일럿,"파일럿 오너입니다. 패밀리카로는 정말 최고입니다. 넓은 공간, 조용하고 묵직한 주행...",3,1,0
120484,혼다,파일럿,제가 아는 현대차 카마스터님과 말투가 너무 비슷하시네요 파일럿 최고죠 할인해줬던 가...,2,1,1
120485,혼다,파일럿,혼다 변속기 버튼은 헷갈리지 않습니다. 직관적입니다. 현대전자식 변속기가 배워야 할...,4,1,0
120486,혼다,파일럿,이전에 F10520D 타 신서 기억합니다. 그 이후에 F10520D를 구매해서 타고...,2,1,0


## 데이터 전처리

In [8]:
# youtube_df['comments'] = youtube_df['comments'].str.replace("[^A-Za-z0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# youtube_df['comments']

In [9]:
# 3036개 - 10분 58초
# 전처리 파일 완성되면 코드 삭제

# import re

# words = youtube_df['comments']

# for i in range(len(words)):
    
#   spelled_sent = spell_checker.check(youtube_df['comments'][i]) # 맞춤법 검사
#   hanspell_sent = spelled_sent.checked # 띄어쓰기 교정
#   result = " ".join(hanspell_sent.split())
#   youtube_df['comments'][i] = result
  
#   print(result)


# youtube_df

In [10]:
# 라벨을 int -> str으로 바꾸기
youtube_df['related'] = youtube_df['related'].astype(str)
youtube_df['emotion'] = youtube_df['emotion'].astype(str)

In [11]:
# 공백으로 된 행 nan 값으로 바꿔주기
youtube_df['comments'] = youtube_df['comments'].replace('', np.nan)

In [12]:
# 공백으로 된 행 갯수
youtube_df['comments'].isnull().sum()

305

In [13]:
# 중립 행 없애기
youtube_df['emotion'] = youtube_df['emotion'].replace('-1', np.nan)

In [14]:
youtube_df['emotion'].isnull().sum()

1943

In [15]:
# nan 값 삭제
youtube_df = youtube_df.dropna(how = 'any')
youtube_df

,car_brand,car_model,comments,likes,related,emotion
0,BMW,1시리즈,"주제 선정 좋아요. 더운 여름 잘 지내세요. 비싼 차, 싼 차, 슈퍼카, 경차 가리...",3,1,0
1,BMW,1시리즈,주차장시승평잘봤네요ㅋ그냥외관이랑실내평만보면편안할듯ㅋㅋ매력적인차네요세컨으로,3,1,0
2,BMW,1시리즈,차 가격도 싸고 디자인도 예쁘고 엄청 좋다,5,1,0
3,BMW,1시리즈,좀끄럽다하셨는데G305시리즈랑어느정도차이가날까요?ㅎㅎ,2,1,0
4,BMW,1시리즈,그나마 전륜으로 바뀌면서 공간이 좀 쓸 만해졌죠. 이 전 세대 1시리즈는 정말......,2,1,1
...,...,...,...,...,...,...
120483,혼다,파일럿,"파일럿 오너입니다. 패밀리카로는 정말 최고입니다. 넓은 공간, 조용하고 묵직한 주행...",3,1,0
120484,혼다,파일럿,제가 아는 현대차 카마스터님과 말투가 너무 비슷하시네요 파일럿 최고죠 할인해줬던 가...,2,1,1
120485,혼다,파일럿,혼다 변속기 버튼은 헷갈리지 않습니다. 직관적입니다. 현대전자식 변속기가 배워야 할...,4,1,0
120486,혼다,파일럿,이전에 F10520D 타 신서 기억합니다. 그 이후에 F10520D를 구매해서 타고...,2,1,0


In [16]:
youtube_df = youtube_df.reset_index()

In [17]:
youtube_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118260 entries, 0 to 118259
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   index      118260 non-null  int64 
 1   car_brand  118260 non-null  object
 2   car_model  118260 non-null  object
 3   comments   118260 non-null  object
 4   likes      118260 non-null  int64 
 5   related    118260 non-null  object
 6   emotion    118260 non-null  object
dtypes: int64(2), object(5)
memory usage: 6.3+ MB


## 브랜드명, 차량 모델명 리스트에 담기

In [18]:
# 브랜드명 리스트에 담기
car_brand_lst = list(youtube_df['car_brand'].value_counts().index)
car_brand_lst

['현대',
 '기아',
 '한국GM',
 '벤츠',
 '르노코리아',
 'BMW',
 '아우디',
 '폭스바겐',
 '쌍용',
 '포드',
 '포르쉐',
 '도요타',
 '볼보',
 '재규어',
 '닛산',
 '혼다',
 '랜드로버',
 '푸조',
 '시트로엥',
 '미쯔비시',
 '롤스로이스',
 '기타',
 '마쯔다',
 '피아트',
 '크라이슬러',
 '사브',
 '다이하쓰']

In [19]:
# 차량 모델 리스트에 담기
car_model_lst = list(youtube_df['car_model'].value_counts().index)
car_model_lst

['기타',
 '트위지',
 '쏘나타',
 'XM3',
 '아반떼',
 '다마스',
 'F150',
 'K5',
 '윈드스타',
 '싼타페',
 '투싼',
 '타이칸',
 '제네시스',
 'S-클래스',
 '보라',
 '캐스퍼',
 '포터',
 'K8',
 '모닝',
 '무쏘',
 'iX',
 '제타',
 'X5',
 '스타렉스',
 'K9',
 '쏘렌토',
 'EV6',
 '아이오닉',
 '레토나',
 '니로',
 '카니발',
 '쉐보레 말리부',
 'SM6',
 '쉐보레 트레일블레이저',
 '코나',
 '아이오닉5',
 '쉐보레 스파크',
 '셀토스',
 '티볼리',
 '스포티지',
 '스팅어',
 '모하비',
 'K7',
 '스토닉',
 '그랜저',
 '체어맨',
 '쉐보레 트래버스',
 '레인지로버 벨라',
 '코란도',
 '오피러스',
 '5시리즈',
 'BB',
 '봉고',
 '4시리즈',
 'G-클래스 (G바겐)',
 '벨로스터',
 '옵티마',
 'Q8',
 '아슬란',
 '투아렉',
 '쉐보레 콜벳',
 'EQA',
 '스타리아',
 '카렌스',
 'A5',
 'i30',
 'A7',
 '클리오',
 '마칸',
 'A6',
 'GLC-클래스',
 'XC90',
 'K3',
 '마치',
 '아테온',
 '어코드',
 'i40',
 'C-클래스',
 'A4',
 '골프',
 '파나메라',
 '캡처',
 'E-Series',
 '토스카',
 '911',
 '티뷰론',
 '알페온',
 '쏘울',
 '206',
 'A-클래스',
 'F-TYPE',
 '파제로',
 '엘란',
 '쉐보레 볼트',
 '수프라',
 'X6',
 'M시리즈',
 'RS Q8',
 '승용차',
 '버스(26인이상)',
 '6시리즈',
 '3시리즈',
 '2008',
 '쉐보레 크루즈',
 'GL-클래스',
 '펠리세이드',
 '머스탱',
 'S7',
 'CLS-클래스',
 'X3',
 'RS7',
 'QM6',
 'S90',
 '쉐보레 콜로라도',
 '제네시스쿠페',
 '쉐보레 

In [20]:
car_lst = list(youtube_df[['car_brand','car_model']].value_counts().index)
car_lst

[('르노코리아', '트위지'),
 ('현대', '기타'),
 ('현대', '쏘나타'),
 ('쌍용', '기타'),
 ('르노코리아', 'XM3'),
 ('현대', '아반떼'),
 ('한국GM', '다마스'),
 ('포드', 'F150'),
 ('기아', 'K5'),
 ('포드', '윈드스타'),
 ('현대', '싼타페'),
 ('포르쉐', '타이칸'),
 ('현대', '투싼'),
 ('현대', '제네시스'),
 ('벤츠', 'S-클래스'),
 ('폭스바겐', '보라'),
 ('현대', '캐스퍼'),
 ('현대', '포터'),
 ('기아', 'K8'),
 ('기아', '모닝'),
 ('쌍용', '무쏘'),
 ('BMW', 'iX'),
 ('폭스바겐', '제타'),
 ('BMW', 'X5'),
 ('현대', '스타렉스'),
 ('기아', 'K9'),
 ('기아', '쏘렌토'),
 ('기아', 'EV6'),
 ('현대', '아이오닉'),
 ('기아', '레토나'),
 ('기아', '니로'),
 ('기아', '카니발'),
 ('한국GM', '쉐보레 말리부'),
 ('르노코리아', 'SM6'),
 ('한국GM', '쉐보레 트레일블레이저'),
 ('현대', '코나'),
 ('현대', '아이오닉5'),
 ('한국GM', '쉐보레 스파크'),
 ('기아', '셀토스'),
 ('쌍용', '티볼리'),
 ('기아', '스포티지'),
 ('기아', '스팅어'),
 ('포드', '기타'),
 ('벤츠', '기타'),
 ('기아', '모하비'),
 ('기아', 'K7'),
 ('기아', '스토닉'),
 ('현대', '그랜저'),
 ('쌍용', '체어맨'),
 ('한국GM', '쉐보레 트래버스'),
 ('랜드로버', '레인지로버 벨라'),
 ('쌍용', '코란도'),
 ('기아', '오피러스'),
 ('BMW', '5시리즈'),
 ('도요타', 'BB'),
 ('기아', '봉고'),
 ('BMW', '4시리즈'),
 ('벤츠', 'G-클래스 (G바겐)'),
 ('현대', '벨로스터'

In [21]:
len(car_lst)

419

In [23]:
youtube_df[(youtube_df['car_brand'].str.contains('현대')&youtube_df['car_model'].str.contains('트럭(2.5~4.5톤)'))]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,index,car_brand,car_model,comments,likes,related,emotion


In [ ]:
youtube_df

,index,car_brand,car_model,comments,likes,related,emotion
0,0,BMW,1시리즈,"주제 선정 좋아요. 더운 여름 잘 지내세요. 비싼 차, 싼 차, 슈퍼카, 경차 가리...",3,1,0
1,1,BMW,1시리즈,주차장시승평잘봤네요ㅋ그냥외관이랑실내평만보면편안할듯ㅋㅋ매력적인차네요세컨으로,3,1,0
2,2,BMW,1시리즈,차 가격도 싸고 디자인도 예쁘고 엄청 좋다,5,1,0
3,3,BMW,1시리즈,좀끄럽다하셨는데G305시리즈랑어느정도차이가날까요?ㅎㅎ,2,1,0
4,4,BMW,1시리즈,그나마 전륜으로 바뀌면서 공간이 좀 쓸 만해졌죠. 이 전 세대 1시리즈는 정말......,2,1,1
...,...,...,...,...,...,...,...
118255,120483,혼다,파일럿,"파일럿 오너입니다. 패밀리카로는 정말 최고입니다. 넓은 공간, 조용하고 묵직한 주행...",3,1,0
118256,120484,혼다,파일럿,제가 아는 현대차 카마스터님과 말투가 너무 비슷하시네요 파일럿 최고죠 할인해줬던 가...,2,1,1
118257,120485,혼다,파일럿,혼다 변속기 버튼은 헷갈리지 않습니다. 직관적입니다. 현대전자식 변속기가 배워야 할...,4,1,0
118258,120486,혼다,파일럿,이전에 F10520D 타 신서 기억합니다. 그 이후에 F10520D를 구매해서 타고...,2,1,0


In [ ]:
car_youbute_df = youtube_df[(youtube_df['car_brand'].str.contains(car_lst[119][0])&youtube_df['car_model'].str.contains(car_lst[119][1]))]
car_youbute_df

,index,car_brand,car_model,comments,likes,related,emotion
21696,22105,기아,엔터프라이즈,저 당시에 저런 옵션들이 있었다는 게 정말 놀랍네요. 기아에서 정말 정성 들여서 만...,13,1,0
21697,22106,기아,엔터프라이즈,저희 아버지의 첫 자동차인 대우 누비라 리뷰 부탁드립니다 레간자를 타고 싶다고 했는...,37,1,0
21698,22107,기아,엔터프라이즈,정말 초기형 모델로 관리 잘 된 모델로 해주셨어감 사하네요. 중학교 때 아버지랑 화...,16,1,0
21699,22108,기아,엔터프라이즈,3.0SL4년 정도 탔었는데 당시 상당히 좋았었죠 특이 디지털 계기판!! 아직도 저...,1,1,0
21700,22109,기아,엔터프라이즈,불과6년전까지타고다녔던아부지차이네요ㅠㅠ승차감은아직도넘사벽이라고느낌니다ㅠㅠ그립다,11,1,0
...,...,...,...,...,...,...,...
22046,22457,기아,엔터프라이즈,머야 장난하냐 19이라고? 차가 너보다 연식이 더 오래됐는데 건방지게 타고 다니네,2,1,1
22047,22458,기아,엔터프라이즈,JMTV 매일매일 잘 보고 있습니다. 99년식 백 엔터 3.0CEO 오너였고 동호회...,3,1,0
22048,22459,기아,엔터프라이즈,06년 초등학교 1학년 때 아버지가 3.0CEO로 이 차량을 탔던 게 기억이 나네요...,2,1,0
22049,22460,기아,엔터프라이즈,이차를 내가 타던 게 껌 좀 씹을 때 승차감 기름 많이 먹음,2,1,0


# 브랜드, 차량 모델별로 for문 돌리면서 워드 클라우드 생성


In [ ]:
# 폰트 다운
%config InlineBackend.figure_format = 'retina'

!apt -qq -y install fonts-nanum > /dev/null

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)

In [ ]:
# 자동차 이미지로 출력하기 위헤 흑백이미지를 백터화
from PIL import Image
import numpy as np

img_array = np.array(Image.open('car_image.png').convert('L'))

for i in img_array:
  print(i)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255]
[255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 253 178  77   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0  63 160 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 202  93   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
 101 215 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 252 122   9   0   0   0   0   0   0   0   0
   0   0   0   0   0   0  

In [ ]:
# 불용어 설정
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','듯','것','저','ㅋㅋ','수','ㅎㅎ','ㅋㅋㅋㅋㅋ','ㅋㅋㅋ','ㅋㅋㅋㅋ','ㅋ','나','때','데','지','현대','기아','한국GM','벤츠','르노코리아','BMW','아우디','폭스바겐','쌍용','포드','포르쉐','도요타','볼보','재규어','닛산','혼다','랜드로버',
 '푸조','시트로엥','미쯔비시','롤스로이스','기타','마쯔다','피아트','크라이슬러','사브','다이하쓰','기타','트위지','쏘나타','XM3','아반떼','다마스','F150','K5','윈드스타','싼타페','제네시스','타이칸','투싼','S-클래스','캐스퍼','보라','포터','K8','모닝','제타','iX','X5','무쏘','스타렉스','쏘렌토','EV6','K9','아이오닉','레토나','쉐보레 말리부','카니발',
 '니로','쉐보레 트레일블레이저','SM6','코나','아이오닉5','셀토스','티볼리','쉐보레 스파크','스팅어','스포티지','모하비','K7','그랜저','스토닉','체어맨','쉐보레 트래버스','레인지로버 벨라','코란도','5시리즈','오피러스','BB','벨로스터','봉고','4시리즈','G-클래스 (G바겐)','옵티마','Q8','투아렉','쉐보레 콜벳','EQA','스타리아','아슬란','i30','A5',
 '카렌스','클리오','A7','GLC-클래스','A6','마칸','XC90','K3','아테온','어코드','마치','i40','C-클래스','A4','골프','파나메라','캡처','E-Series','토스카','911','티뷰론','쏘울','A-클래스','알페온','파제로','206','F-TYPE','수프라','엘란','쉐보레 볼트','X6','3시리즈','버스(26인이상)','M시리즈','2008','RS Q8','GL-클래스','승용차',
 '머스탱','펠리세이드','6시리즈','쉐보레 크루즈','X3','CLS-클래스','S7','RS7','QM6','S90','레인지로버 이보크','제네시스쿠페','쉐보레 트랙스','쉐보레 콜로라도','쉐보레 카마로','아카디아','트럭(4.5톤이상)','엔터프라이즈','DS7','Z4','T-Roc','포니','1시리즈','SLC-클래스','e-tron','XC60','GLS-클래스','오딧세이','C30','고스트','엘란트라','CR-V',
 '프리우스','엑스트렉','트럭','티구안','XF','로체','랜서 에볼루션','티코','A8','SL-클래스','쉐보레 이쿼녹스','C3','XC40','디스커버리 스포츠','CLA-클래스','마크X','Q3','세레스','XJ-6','I-PACE','Q2','AMG GT','R8','엑셀','다이너스티','2시리즈','갤로퍼','디펜더','넥쏘','X7','카이엔','조에','SM7','Q5','S70','파사트','포르테','맥스크루즈',
 '박스터','CL-클래스','레이','CLK-클래스','크레도스','8시리즈','시로코','208','스카이라인','RAV 4','S6','X4','디스커버리','GT-R','86','508','CX-5','MX-5 MIATA','GLE-클래스','트럭(2.5~4.5톤)','RS5','리오','에쿠스','쉐보레 올란도','레이스','3008','던','A3','EOS','XJ-8','406','렉스턴','리지라인','알티마','파일럿','쉐보레 캡티바',
 '버스(16~36인)','GLB-클래스','마이티','V50','DS4','시에나','S60','i8','쏠라티','9-3','쉐보레 임팔라','액티언''트레일러','S-TYPE','SM5','C5','QM3','델솔','마테리아','SLK-클래스','라보','S8','매그너스','아발론','이스타나','124','투스카니','브로엄','리베로','엑센트','프론티어','레인지로버','Q7','카이맨','리프','DS3','X1','몬데오','비스토',
 'A1','500','시빅','S3','V90','스프린터','XJR','캐러밴','마스터','쉐보레 아베오','5008','SM3','레인지로버 스포츠','X2','i3','XE','SQ5','C4','레인저','V60','E-PACE','패스파인더','SQ8','EQC','카이런','타이탄','WiLL','코펜','X-Trail','S660','FJ 크루져','그란투리스모 (GT)','N Box','XKR''캠리','마르샤','QM5','캐피탈','로디우스','테라칸',
 '포텐샤','이클립스','200','308','휘가로','베리타스','F-PACE','500X ','스텔라','라세티','그레이스','폴로','쿠가','HR-V','TT','NV','익스플로러','비틀','RVR','싼타모','세피아','마티즈','V40','GLA-클래스','툰드라','파쏘','익스피디션','맥시마','대우버스','370Z','9-5','페이톤','V-클래스','B-클래스 (MY B)','트라제XG','레전드','클릭','엘그란드',
 '7시리즈','스펙트라','익스플로러 스포츠트랙','X-TYPE','무라노','CC','엑시언트','하이랜더','207','S4','G2X','트라고','C70','트랜짓','타코마','베르나','솔라라','프라이드','쎄라토','엘리멘트','9000','XC70','900','300C','PT 크루저','S-MAX','베라크루즈','르망','라비타','윈스톰','레조','라노스','프레스토','티아나','센트라','607','타우너',
 'TTS','307','큐브','프런티어','M-클래스','벤자','소버린','미라코코아','아웃랜더','쥬크','프린스','407','GLK-클래스','젠트라','굴삭기','크로스파이어','RCZ','S5','XJ','파이브 헌드레드','그랜드 보이저','프리랜더','이스케이프','퓨전','미라지노','S40','프레지오','카스타','모코','타운 앤 컨트리','랜서','S80','R-클래스','올로드 콰트로','스테이츠맨','크로스투어',
 '코롤라','퍼시피카','토러스','세브링','프리몬트','DS5','인사이트','포커스','로그','차','단점','장점','근데','이거','분','비','저거','정도','나','말','ㅎ','되','때문','내','ㅎㅎㅎㅎㅎ','지금','개','점','타고','편','저희','뭐','저차','그','우','바','만','라','님','으','ㄷㄷㄷ','ㅠ','ㅜ','등','얘','보','그거','ㅎㅎㅎㅎ','ㅎㅎ저','기','ㅠㅠㅠ','타','후',
 '시발','쌍','차','것','수','진짜','코','리','때','적','ㅎㅎ','지','정도','작','라','오','이젠','보','바','만','쪽','나름','안되서','내','안','이거','대','뿐','ㅠ','두','되길','뭔가','그걸','드','왜이렇게','분','ㄷㄷ','조금더','평소','ㄷ','행','되겼음','어찌됫건','유','하지만지금까지','님','전','레','G80','ㅎㅎㅎ','쉐보레','ㅠㅠ','식','G90','G70','전',
 'GV80','이번','소나타','GV70','거','마스','위','형','중','엠','우리','시','2','3','세','와우','^ㅎ^ㅎ','변','터','솔','추','모','사실','아우','노','번','무','4.','1.','년','김한용','3.','십','^ㅋ','^ㅎ','^','5','2.','면','1.','^ㅎ^ㅎ^ㅎ','등등','용','제','^ㅋ^ㅋ^ㅋ','^ㅋ^ㅋ^ㅋ^ㅋ^ㅋ','어디','조','ㅂ','닥','징','란','거에요','그다음','이하',
 '음','로','키','정말','제','일','함','더','임','게','보고','언제','역시','무조건','요즘','오늘','가끔','가장','다만','완전','요','그냥','난','꼭','알','항상','바로','승','살','얼마나','너','사고','및','임','함','왜','용','니','트','토','현재','대해','점점','은근','욯','저번','오오','오오오','요요','음','옹','리뷰','솔님','누구','뒈','조금','또','아주','덕분',
 '영상','이제','그때','몇','김기태','어','는걸', '명','이건','당장','못','디','꽤','저런','제발','날','성','새','무엇','당시','배','구']

# 고려할 단어
# '리뷰', '기자님','영상'

okt = Okt()
tokenizer = Tokenizer()

In [ ]:
# 긍정

positive_han_nouns = []


for i in range(len(car_lst)):
  car_youbute_df = youtube_df[(youtube_df['car_brand'].str.contains(car_lst[i][0])&youtube_df['car_model'].str.contains(car_lst[i][1]))]

  positive_han_nouns = []

  # 감성 분류 긍정(0)이면 -> 워드 클라우드 생성
  positive_df = car_youbute_df[(car_youbute_df['emotion'].str.contains('0'))]
  print(car_lst[i][0], car_lst[i][1])
  print(len(positive_df))
  try:
    for p_comment in positive_df['comments']:
      tokenized_sentence_p = okt.nouns(p_comment)
      stopwords_removed_sentence_p = [word for word in tokenized_sentence_p if not word in stopwords] # 불용어 제거
      positive_han_nouns.append(stopwords_removed_sentence_p)

      # 글자수 한자리 단어, 'ㅋ'이 포함된 단어은 불용어 사전에 넣기
    #   for j in stopwords_removed_sentence_p:
    #     if len(j) <= 1 or 'ㅋ' or 'ㅠ' or 'ㅡ' or 'ㅎ' in j:
    #       stopwords.append(j)
    # print(stopwords)

    word_list_p = sum(positive_han_nouns, [])
    count_p = Counter(word_list_p)
    word_count_p = dict(count_p.most_common())
    print(word_count_p)

    # 로컬에 있는 폰트를 사용할 경우, 폰트의 경로를 font_path에 추가 해주시면 됩니다.
    wc = WordCloud(font_path=fontpath, background_color = 'white',colormap=matplotlib.cm.ocean, width=800, height=800, min_font_size = 8, max_font_size=120, prefer_horizontal = True, mask=img_array)
    cloud_p = wc.fit_words(word_count_p)
    cloud_p.to_image()
    cloud_p.to_file(f'{car_lst[i][0]}+{car_lst[i][1]}+긍정.jpg')
  except:
    print(car_lst[i][0], car_lst[i][1])
    pass

르노코리아 트위지
821
{'트위': 138, '창문': 71, '오토바이': 66, '생각': 60, '주행': 50, '주차': 44, '에어컨': 42, '충전': 37, '차량': 36, '거리': 35, '가격': 33, '사람': 32, '전기차': 32, '출퇴근': 31, '구매': 28, '레이첼': 25, '전기': 23, '원': 22, '겨울': 21, '이차': 20, '속도': 19, '자동차': 19, '경차': 18, '배달': 18, '사용': 18, '걸': 18, '뒤': 18, '여름': 18, '차감': 18, '스쿠터': 17, '히터': 17, '회사': 16, '운전': 16, '시트': 15, '걱정': 15, '휠체어': 15, '배터리': 13, '최고': 13, '벨로스': 13, '바이크': 13, '문제': 13, '중고차': 13, '앞': 13, '중고': 13, '이면': 13, '킬로': 12, '곳': 12, '차라': 12, '킥': 12, '보드': 12, '하나': 12, '장님': 12, '디스크': 12, '비교': 11, '가지': 11, '고민': 11, '옵션': 11, '차로': 11, '자리': 11, '건': 11, '느낌': 11, '도로': 11, '스파크': 11, '한번': 11, '실용': 10, '동네': 10, '목소리': 10, '비닐': 10, '륜': 10, '용도': 10, '목숨': 10, '허리': 10, '설명': 10, '기자': 9, '처음': 9, '달': 9, '은진': 9, '지붕': 9, '아파트': 9, '전동': 9, '관심': 9, '버스': 9, '야근': 9, '돈': 9, '말씀': 9, '승기': 8, '주차장': 8, '여자': 8, '시골': 8, '산다': 8, '디자인': 8, '건가': 8, '혼자': 8, '선택': 8, '카트': 8, '유리': 8, '편의': 8, '노트': 8, '소리': 8, '날씨': 8, '이

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys


벤츠 G-클래스 (G바겐)
0
{}
벤츠 G-클래스 (G바겐)
현대 벨로스터
274
{'벨로스': 93, '벨': 28, '디자인': 26, '오토': 25, '생각': 22, '가격': 22, '차량': 20, '뷰': 20, '터보': 19, '기자': 18, '비교': 17, '성능': 16, '승기': 15, '수동': 15, '느낌': 14, '고성능': 13, '일리': 13, '모델': 12, '실내': 12, '옵션': 11, '사람': 11, '기대': 11, '서킷': 11, '부분': 10, '스포츠': 10, '주행': 10, '카': 9, '성비': 9, '현기': 9, '건': 9, '브랜드': 9, '선택': 8, '운전': 8, '개인': 8, '중형': 8, '관심': 8, '걸': 8, '재미': 8, '마력': 8, '상기': 7, '준': 7, '추천': 7, '엔진': 7, '구매': 6, '최고': 6, '한번': 6, '굳이': 6, '시트': 6, '사운드': 6, '가지': 6, '현대차': 6, '처음': 6, '세대': 6, '고민': 6, '티': 6, '구입': 6, '만족': 6, '코너링': 6, '가요': 6, '대비': 6, '차감': 6, '해치백': 6, '가치': 5, '한상기': 5, '출력': 5, '해도': 5, '대부분': 5, '코너': 5, '형님': 5, '자동차': 5, '문제': 5, '중고': 5, '줄': 5, '미국': 5, '크루즈': 5, '매력': 4, '펀': 4, '드라이빙': 4, '다시': 4, '세단': 4, '대가': 4, '대한': 4, '실제': 4, '이상': 4, '출시': 4, '소재': 4, '뒤': 4, '자리': 4, '기준': 4, '번호판': 4, '다른': 4, '애': 4, '앞': 4, '그룹': 4, '라인': 4, '차주': 4, '쉐슬': 4, '칭찬': 3, '수동변속기': 3, '로스': 3, '홀드': 3, '탈': 3, '세

In [ ]:
# 부정

nagative_han_nouns = []
# stopwords_removed_sentence_n = []

for i in range(len(car_lst)):
  car_youbute_df = youtube_df[(youtube_df['car_brand'].str.contains(car_lst[i][0])&youtube_df['car_model'].str.contains(car_lst[i][1]))]

  nagative_df = car_youbute_df[(car_youbute_df['emotion'].str.contains('1'))]
  print(car_lst[i][0], car_lst[i][1])
  print(len(nagative_df))

  try:
    for n_comment in nagative_df['comments']:
      tokenized_sentence_n = okt.nouns(n_comment)
      # print(tokenized_sentence_n, n_comment)
      # print(tokenized_sentence_n, n_comment)

      # for word in tokenized_sentence_n: 
      #   if len(word) <= 1:
      #     stopwords.append(word)
      #   else:
      #     if word not in stopwords:
      #       stopwords_removed_sentence_n.append(word) 
      stopwords_removed_sentence_n = [word for word in tokenized_sentence_n if not word in stopwords] # 불용어 제거
      nagative_han_nouns.append(stopwords_removed_sentence_n)


      # print(nagative_han_nouns)

      # 글자수 한자리 단어, 'ㅋ'이 포함된 단어은 불용어 사전에 넣기
      # for z in stopwords_removed_sentence_n:
      #   if len(z) <= 1 or 'ㅋ' or 'ㅠ' or 'ㅡ' or 'ㅎ' in j:
      #     stopwords.append(z)

    word_list_n = sum(nagative_han_nouns, [])
    count_n = Counter(word_list_n)
    word_count_n = dict(count_n.most_common())
    print(word_count_n)

    # 로컬에 있는 폰트를 사용할 경우, 폰트의 경로를 font_path에 추가 해주시면 됩니다.
    wc = WordCloud(font_path=fontpath, background_color = 'white',colormap=matplotlib.cm.gist_heat, min_font_size = 8, max_font_size=120, width=800, height=800, prefer_horizontal = True, mask=img_array)
    cloud_n = wc.fit_words(word_count_n)
    cloud_n.to_image()
    cloud_n.to_file(f'{car_lst[i][0]}+{car_lst[i][1]}+부정.jpg')
  except:
    print(car_lst[i][0], car_lst[i][1])
    pass

Output hidden; open in https://colab.research.google.com to view.